In [4]:
import ee
import zipfile
import geopandas as gpd
import ee
import pandas as pd

ee.Initialize()
ee.Authenticate

<function ee.Authenticate(authorization_code: Optional[str] = None, quiet: Optional[bool] = None, code_verifier: Optional[str] = None, auth_mode: Optional[str] = None, scopes: Optional[Sequence[str]] = None, force: bool = False) -> Optional[bool]>

In [5]:
shapefile_path = "C:/Users/pc/My Drive/2025/Uni/TCC/Shapes/contorno_area_total/contorno_area_total.shp"

# Check if the path is a .zip file
if shapefile_path.endswith('.zip'):
    # Try to read shapefile from a zip archive
    try:
        # Check if the .zip file exists and open it
        with zipfile.ZipFile(shapefile_path, 'r') as zip_ref:
            zip_ref.printdir()  # Optional: Print contents of the zip to debug
            # Try to find the .shp file inside the zip
            shapefile_found = False
            for file in zip_ref.namelist():
                if file.endswith('.shp'):
                    shapefile_found = True
                    shapefile_within_zip = file
                    break

            if shapefile_found:
                # Read shapefile directly from the zip file
                oi = gpd.read_file(f'zip://{shapefile_path}/{shapefile_within_zip}')
                print(f"Successfully loaded shapefile from {shapefile_path}.")
            else:
                print("No .shp file found inside the zip archive.")
                #
    except Exception as e:
        print(f"Error reading shapefile from zip archive: {e}")
        #
else:
    # If not a .zip, assume it is a regular shapefile
    try:
        # Read the shapefile normally
        aoi = gpd.read_file(shapefile_path)
        print(f"Successfully loaded shapefile from {shapefile_path}.")
    except Exception as e:
        print(f"Error reading shapefile: {e}")


# After loading, check if the GeoDataFrame is not empty
if not aoi.empty:
    # If the GeoDataFrame contains multiple geometries, dissolve them into one
    if len(aoi) > 1:
        aoi = aoi.dissolve()

    # Extract the first geometry from the dissolved GeoDataFrame
    geometry = aoi.geometry.iloc[0]

    # Check if the geometry is a Polygon or MultiPolygon
    if geometry.geom_type in ['Polygon', 'MultiPolygon']:
        # Convert the geometry to GeoJSON format
        geojson = geometry.__geo_interface__

        # Remove the third dimension from the coordinates if it exists
        if geojson['type'] == 'Polygon':
            geojson['coordinates'] = [list(map(lambda coord: coord[:2], ring)) for ring in geojson['coordinates']]
        elif geojson['type'] == 'MultiPolygon':
            geojson['coordinates'] = [[list(map(lambda coord: coord[:2], ring)) for ring in polygon] for polygon in geojson['coordinates']]

        # Create an Earth Engine geometry object from the GeoJSON coordinates
        ee_geometry = ee.Geometry(geojson)

        # Convert the Earth Engine geometry to a Feature
        feature = ee.Feature(ee_geometry)

        # Create a FeatureCollection with the feature
        aoi = ee.FeatureCollection([feature])

        print("AOI defined successfully.")

        # check_next_button()
    else:
        
        print("The geometry is not a valid type (Polygon or MultiPolygon).")
else:
    print("The shapefile does not contain any geometries.")        #

Successfully loaded shapefile from C:/Users/pc/My Drive/2025/Uni/TCC/Shapes/contorno_area_total/contorno_area_total.shp.
AOI defined successfully.


In [6]:
# Define the start and end dates for filtering the image collection
inicio = '2022-01-01'
final = '2023-12-31'
nuvem ='40'

# Define the start and end dates for filtering the image collection
startDate = '2022-01-01'
endDate = '2023-12-31'

# Load the Sentinel-2 image collection and filter by date, location, and cloud coverage
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(startDate, endDate) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))


mask = True
if mask:
    def mask_cloud_and_shadows(image):
        # Check if the cloud probability and snow probability bands are available
        band_names = image.bandNames()
        has_cloud_prob = band_names.contains('MSK_CLDPRB')
        has_snow_prob = band_names.contains('MSK_SNWPRB')

        # Create masks based on the availability of the bands
        cloud_mask = ee.Image(1) if not has_cloud_prob else image.select('MSK_CLDPRB').lt(1)
        snow_mask = ee.Image(1) if not has_snow_prob else image.select('MSK_SNWPRB').lt(1)

        # Scene classification layer
        scl = image.select('SCL')
        shadow_mask = scl.neq(3)  # Shadow class
        cirrus_mask = scl.neq(10)  # Cirrus class

        # Combine all masks
        mask = cloud_mask.And(snow_mask).And(shadow_mask).And(cirrus_mask)
        return image.updateMask(mask)

    # Apply the cloud and shadow mask function to the image collection
    sentinel = sentinel2.map(mask_cloud_and_shadows)

In [7]:
#Coverage Ratio Function
# -------------------------------
aoi_geometry = aoi.first().geometry()
aoi_area = aoi_geometry.area()

coverage_threshold = 0.9

def calculate_coverage_ratio(image):
    """
    Calculates the ratio of the AOI area covered by the image.
    
    Args:
        image (ee.Image): The Sentinel-2 image.
    
    Returns:
        ee.Image: The original image with an added 'coverage_ratio' property.
    """
    # Compute the intersection geometry between AOI and image footprint
    intersection = aoi_geometry.intersection(image.geometry(), ee.ErrorMargin(1))
    
    # Calculate the area of the intersection
    intersection_area = intersection.area()
    
    # Calculate the coverage ratio (intersection area / AOI area)
    coverage_ratio = intersection_area.divide(aoi_area)
    
    # Set the coverage ratio as a property of the image
    return image.set('coverage_ratio', coverage_ratio)

# -------------------------------
# Step 6: Apply Coverage Ratio Calculation
# -------------------------------

# Map the coverage ratio function over the Sentinel-2 collection
sentinel2_with_ratio = sentinel2.map(calculate_coverage_ratio)

# -------------------------------
# Step 7: Filter Based on Coverage Ratio
# -------------------------------

# Define a filter to keep images with coverage_ratio >= coverage_threshold
coverage_filter = ee.Filter.gte('coverage_ratio', coverage_threshold)

# Apply the filter to get the final collection
fully_covering_images = sentinel2_with_ratio.filter(coverage_filter)

# Get the number of images before filtering
initial_count = sentinel2.size().getInfo()

# Get the number of images after coverage filtering
filtered_count = fully_covering_images.size().getInfo()

print(f"Number of images before coverage filtering: {initial_count}")
print(f"Number of images with >= {coverage_threshold*100}% AOI coverage: {filtered_count}")

Number of images before coverage filtering: 297
Number of images with >= 90.0% AOI coverage: 278


In [8]:
sentine2 = filtered_count

In [9]:
# Define the vegetation index calculation
vegetation_index = 'NDVI'

def calculate_index(image):
    if vegetation_index == 'NDVI':
        index_image = image.normalizedDifference(['B8', 'B4']).rename('index')
    elif vegetation_index == 'EVI':
        index_image = image.expression(
            '2.5 * ((NIR / 10000 - RED / 10000) / (NIR / 10000 + 6 * RED / 10000 - 7.5 * BLUE / 10000 + 1))', {
                'NIR': image.select('B8'),
                'RED': image.select('B4'),
                'BLUE': image.select('B2')
            }
        ).rename('index')
    elif vegetation_index == 'SAVI':
        L = 0.5
        index_image = image.expression(
            '(1 + L) * ((NIR / 10000) - (RED / 10000)) / ((NIR / 10000) + (RED / 10000) + L)', {
                'NIR': image.select('B8'),
                'RED': image.select('B4'),
                'L': L
            }
        ).rename('index')
    elif vegetation_index == 'GCI':
        index_image = image.expression(
            'NIR / GREEN - 1', {
                'NIR': image.select('B8'),
                'GREEN': image.select('B3')
            }
        ).rename('index')
    elif vegetation_index == 'GNDVI':
        index_image = image.normalizedDifference(['B8', 'B3']).rename('index')
    else:
        raise ValueError(f"Unsupported vegetation index: {vegetation_index}")

    # Calculate mean value for the index over the buffered AOI
    mean_index = index_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,  # Use the buffered AOI
        scale=10,
        bestEffort=True
    ).get('index')
    
    return image.set({'date': image.date().format('YYYY-MM-dd'), 'mean_index': mean_index})

# Map the calculation and filter results
result = sentinel2.map(calculate_index).filter(ee.Filter.notNull(['mean_index']))
dates = result.aggregate_array('date').getInfo()
mean_indices = result.aggregate_array('mean_index').getInfo()

# Convert to pandas DataFrame
if dates and mean_indices:
    avg_index_values = [{'date': date, 'average_index': index} for date, index in zip(dates, mean_indices)]
    df = pd.DataFrame(avg_index_values)
    print(df)
else:
    print("No data found.")

           date  average_index
0    2022-01-19       0.713293
1    2022-01-19       0.710188
2    2022-01-21       0.865844
3    2022-01-21       0.865615
4    2022-01-26       0.833051
..          ...            ...
292  2023-12-20       0.591922
293  2023-12-25       0.771918
294  2023-12-25       0.771704
295  2023-12-27       0.804468
296  2023-12-27       0.803665

[297 rows x 2 columns]


In [10]:
# Extract the geometry from the FeatureCollection
aoi_geometry = aoi.geometry()

# Buffer the AOI geometry inward by 10 meters (adjust distance as needed)
buffer_distance = -10  # Negative value shrinks the geometry inward
aoi_buffered = aoi_geometry.buffer(buffer_distance)

# Define the vegetation index calculation
vegetation_index = 'NDVI'
vegetation_index = 'NDVI'

def calculate_index(image):
    if vegetation_index == 'NDVI':
        index_image = image.normalizedDifference(['B8', 'B4']).rename('index')
    elif vegetation_index == 'EVI':
        index_image = image.expression(
            '2.5 * ((NIR / 10000 - RED / 10000) / (NIR / 10000 + 6 * RED / 10000 - 7.5 * BLUE / 10000 + 1))', {
                'NIR': image.select('B8'),
                'RED': image.select('B4'),
                'BLUE': image.select('B2')
            }
        ).rename('index')
    elif vegetation_index == 'SAVI':
        L = 0.5
        index_image = image.expression(
            '(1 + L) * ((NIR / 10000) - (RED / 10000)) / ((NIR / 10000) + (RED / 10000) + L)', {
                'NIR': image.select('B8'),
                'RED': image.select('B4'),
                'L': L
            }
        ).rename('index')
    elif vegetation_index == 'GCI':
        index_image = image.expression(
            'NIR / GREEN - 1', {
                'NIR': image.select('B8'),
                'GREEN': image.select('B3')
            }
        ).rename('index')
    elif vegetation_index == 'GNDVI':
        index_image = image.normalizedDifference(['B8', 'B3']).rename('index')
    else:
        raise ValueError(f"Unsupported vegetation index: {vegetation_index}")

    # Calculate mean value for the index over the buffered AOI
    mean_index = index_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi_buffered,  # Use the buffered AOI
        scale=10,
        bestEffort=True
    ).get('index')
    
    return image.set({'date': image.date().format('YYYY-MM-dd'), 'mean_index': mean_index})

# Apply calculations to the existing Sentinel-2 collection
result = sentinel2.map(calculate_index).filter(ee.Filter.notNull(['mean_index']))
dates = result.aggregate_array('date').getInfo()
mean_indices = result.aggregate_array('mean_index').getInfo()

# Convert to pandas DataFrame
if dates and mean_indices:
    avg_index_values = [{'date': date, 'average_index': index} for date, index in zip(dates, mean_indices)]
    df = pd.DataFrame(avg_index_values)
    print(df)
else:
    print("No data found.")

           date  average_index
0    2022-01-19       0.713913
1    2022-01-19       0.710806
2    2022-01-21       0.868187
3    2022-01-21       0.867953
4    2022-01-26       0.834943
..          ...            ...
292  2023-12-20       0.594253
293  2023-12-25       0.775591
294  2023-12-25       0.775375
295  2023-12-27       0.808138
296  2023-12-27       0.807319

[297 rows x 2 columns]
